<a href="https://colab.research.google.com/github/phatak-dev/transformer-models/blob/main/Spam_Detection_using_BERT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets
!pip install transformers
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
# data from https://www.kaggle.com/datasets/balaka18/email-spam-classification-dataset-csv
file_path="spam.csv"
df = pd.read_csv(file_path,encoding = "ISO-8859-1")

In [5]:
df = df[["v1","v2"]]

In [6]:
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
df['label'] = df.v1.map({'ham':0, 'spam':1})

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [9]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
## test the model accuracy on sample data
#sample_data = df['v2'].to_list()[4000:4100]
#sample_labels = df['label'].to_list()[4000:4100]

In [11]:
#token_sample_output = tokenizer(sample_data,padding=True,return_tensors="pt")
#output_prediction = model(**token_sample_output)
#numpy_output = output_prediction.logits.detach().numpy()
#import numpy as np
#preds = np.argmax(numpy_output, axis=-1)
#from sklearn.metrics import accuracy_score
#accuracy_score(sample_labels, preds)

In [12]:
from datasets import Dataset
train_data_df = df.sample(200, random_state=42)
eval_data_df = df.sample(200, random_state=45)
train_dataset = Dataset.from_pandas(train_data_df)
eval_dataset = Dataset.from_pandas(eval_data_df)
test_data_df = df.iloc[4000:4100]
test_dataset = Dataset.from_pandas(test_data_df)

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["v2"], padding="max_length", truncation=True)
tokenized_datasets_train = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets_eval = eval_dataset.map(tokenize_function, batched=True)
tokenizer_datasets_test = test_dataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(output_dir="test_trainer")


In [15]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, v2, v1. If __index_level_0__, v2, v1 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 200
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 75
  Number of trainable parameters = 109483778


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=75, training_loss=0.2680803426106771, metrics={'train_runtime': 55.4444, 'train_samples_per_second': 10.822, 'train_steps_per_second': 1.353, 'total_flos': 157866633216000.0, 'train_loss': 0.2680803426106771, 'epoch': 3.0})

In [19]:
from datasets import Dataset


In [21]:
trainer.predict(tokenizer_datasets_test)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: v2, v1. If v2, v1 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100
  Batch size = 8


PredictionOutput(predictions=array([[ 2.9932833 , -2.8134682 ],
       [ 3.1092594 , -3.1133068 ],
       [ 3.1141238 , -2.895039  ],
       [ 2.9373918 , -2.694677  ],
       [ 3.1914847 , -3.0612087 ],
       [ 3.1621814 , -3.0323222 ],
       [ 2.5613303 , -2.797499  ],
       [ 2.980344  , -3.102081  ],
       [ 3.199685  , -3.0958045 ],
       [ 0.58996034, -0.5782189 ],
       [ 0.59334344, -0.5913765 ],
       [ 3.0505834 , -3.0773704 ],
       [ 2.8335729 , -2.9768097 ],
       [ 2.939222  , -3.084695  ],
       [ 2.73827   , -2.9164364 ],
       [ 3.088671  , -3.0031116 ],
       [ 3.1103761 , -2.864685  ],
       [ 3.1172311 , -3.043598  ],
       [ 0.5862061 , -0.57360417],
       [ 1.2876315 , -1.4167782 ],
       [ 3.120337  , -3.0952115 ],
       [ 3.062385  , -2.93528   ],
       [ 3.1467426 , -3.1355784 ],
       [ 3.0128973 , -3.073822  ],
       [ 2.8233976 , -2.9957163 ],
       [ 2.8530865 , -2.9901977 ],
       [ 3.0284903 , -2.742185  ],
       [ 3.0865624 , -2.95